# Android Data from PlayStore

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# Load data
df = pd.read_csv('googleplaystore.csv')
df_user_reviews = pd.read_csv('googleplaystore_user_reviews.csv')
df.columns
df_user_reviews.columns

Index(['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type',
       'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver',
       'Android Ver'],
      dtype='object')

Index(['App', 'Translated_Review', 'Sentiment', 'Sentiment_Polarity',
       'Sentiment_Subjectivity'],
      dtype='object')

In [4]:
# rename columns
df.rename(columns={
    'App': 'name',
    'Category': 'category',
    'Rating': 'rating',
    'Reviews' :'reviews',
    'Size': 'size',
    'Installs': 'installs',
    'Type': 'type',
    'Price': 'price',
    'Content Rating': 'content_rating',
    'Genres': 'genres',
    'Last Updated': 'last_updated',
    'Current Ver': 'version',
    'Android Ver': 'android_version'
    
}, inplace=True)
df.head()

df_user_reviews.rename(
    columns={'App':'app_name', 'Translated_Review': 'review', 'Sentiment': 'sentiment', 'Sentiment_Polarity': 'polarity', 'Sentiment_Subjectivity': 'subjectivity'}
, inplace=True)
df_user_reviews.head()


,name,category,rating,reviews,size,installs,type,price,content_rating,genres,last_updated,version,android_version
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


,app_name,review,sentiment,polarity,subjectivity
0,10 Best Foods for You,I like eat delicious food. That's I'm cooking ...,Positive,1.00,0.533333
1,10 Best Foods for You,This help eating healthy exercise regular basis,Positive,0.25,0.288462
2,10 Best Foods for You,NaN,NaN,NaN,NaN
3,10 Best Foods for You,Works great especially going grocery store,Positive,0.40,0.875000
4,10 Best Foods for You,Best idea us,Positive,1.00,0.300000


In [5]:
orig_df = df.copy()

In [6]:
df = orig_df.copy()

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10841 entries, 0 to 10840
Data columns (total 13 columns):
name               10841 non-null object
category           10841 non-null object
rating             9367 non-null float64
reviews            10841 non-null object
size               10841 non-null object
installs           10841 non-null object
type               10840 non-null object
price              10841 non-null object
content_rating     10840 non-null object
genres             10841 non-null object
last_updated       10841 non-null object
version            10833 non-null object
android_version    10838 non-null object
dtypes: float64(1), object(12)
memory usage: 1.1+ MB


## Preprocessing
Many columns need preformatting to be able to use them in any machine learning models. They should be converted to numbers.

In [8]:
# there are 1181 duplications in the 'name' column
print('Number of duplicate entries : %d' % df.duplicated(subset = ['name']).sum())
# remove duplicates 
df = df[~df.duplicated(subset = ['name'], keep = 'first')]

Number of duplicate entries : 1181


In [9]:
# preformat installs
df = df[df['installs'] != 'Free']
new_df = df['installs'].map(lambda s : s[:-1].replace(',',''))
new_df[new_df == ''] = 0
df['installs'] = new_df.astype(int)
df['installs'].unique()

array([     10000,     500000,    5000000,   50000000,     100000,
            50000,    1000000,   10000000,       5000,  100000000,
       1000000000,       1000,  500000000,         50,        100,
              500,         10,          1,          5,          0])

In [10]:
# preformat reviews
df['reviews'] = df['reviews'].astype(int)
df['reviews'].head()

0       159
1       967
2     87510
3    215644
4       967
Name: reviews, dtype: int64

In [11]:
# preformat size
# np.sort(df['size'].unique())
# transform sizes to kb units
def size_transform(size):
    if size == 'Varies with device':
        return 1
    unit = size[-1]
    number = float(size[:-1])
    if unit == 'M':
        return number * 1000
    if unit == 'k':
        return number
df['size'].head()
df['size'] = df['size'].apply(size_transform).astype(int)
df['size'].head()

0     19M
1     14M
2    8.7M
3     25M
4    2.8M
Name: size, dtype: object

0    19000
1    14000
2     8700
3    25000
4     2800
Name: size, dtype: int64

In [12]:
# preformat price
# df['price'].unique()
temp_df = df['price'].apply(lambda s: s.replace('$', ''))
df['price'] = temp_df.astype(float)
df['price'].unique()

array([  0.  ,   4.99,   3.99,   6.99,   1.49,   2.99,   7.99,   5.99,
         3.49,   1.99,   9.99,   7.49,   0.99,   9.  ,   5.49,  10.  ,
        24.99,  11.99,  79.99,  16.99,  14.99,   1.  ,  29.99,  12.99,
         2.49,  10.99,   1.5 ,  19.99,  15.99,  33.99,  74.99,  39.99,
         3.95,   4.49,   1.7 ,   8.99,   2.  ,   3.88,  25.99, 399.99,
        17.99, 400.  ,   3.02,   1.76,   4.84,   4.77,   1.61,   2.5 ,
         1.59,   6.49,   1.29,   5.  ,  13.99, 299.99, 379.99,  37.99,
        18.99, 389.99,  19.9 ,   8.49,   1.75,  14.  ,   4.85,  46.99,
       109.99, 154.99,   3.08,   2.59,   4.8 ,   1.96,  19.4 ,   3.9 ,
         4.59,  15.46,   3.04,   4.29,   2.6 ,   3.28,   4.6 ,  28.99,
         2.95,   2.9 ,   1.97, 200.  ,  89.99,   2.56,  30.99,   3.61,
       394.99,   1.26,   1.2 ,   1.04])

In [13]:
df['price_rounded'] = df['price'].apply(np.round)

### Categorical values

In [14]:
categories = {}


def categorize(df, columns):
    if isinstance(columns, str):
        column = columns
        df[column], cats = pd.factorize(df[column])
        categories[column] = cats
    else :
        for column in columns:
            df[column], cats = pd.factorize(df[column])
            categories[column] = cats
    return df, cats

In [15]:
# cat_columns = ['type', 'category', 'content_rating', 'genres' ]
# df, _ = categorize(df, cat_columns)

In [16]:
# preformat type
print('Original values : %s' % df['type'].unique())

# there is one app that doesn't have type -> drop it
# df['type'].isnull().sum()
df.dropna(subset=['type'], inplace=True)
# df['type'].isnull().sum()

# convert 'type' column to category
df, _ = categorize(df,'type')
df['type'].unique()
categories['type']

Original values : ['Free' 'Paid' nan]


array([0, 1])

Index(['Free', 'Paid'], dtype='object')

In [17]:
# preformat category
df, _ = categorize(df, 'category')
df['category'].unique()
categories['category']

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32])

Index(['ART_AND_DESIGN', 'AUTO_AND_VEHICLES', 'BEAUTY', 'BOOKS_AND_REFERENCE',
       'BUSINESS', 'COMICS', 'COMMUNICATION', 'DATING', 'EDUCATION',
       'ENTERTAINMENT', 'EVENTS', 'FINANCE', 'FOOD_AND_DRINK',
       'HEALTH_AND_FITNESS', 'HOUSE_AND_HOME', 'LIBRARIES_AND_DEMO',
       'LIFESTYLE', 'GAME', 'FAMILY', 'MEDICAL', 'SOCIAL', 'SHOPPING',
       'PHOTOGRAPHY', 'SPORTS', 'TRAVEL_AND_LOCAL', 'TOOLS', 'PERSONALIZATION',
       'PRODUCTIVITY', 'PARENTING', 'WEATHER', 'VIDEO_PLAYERS',
       'NEWS_AND_MAGAZINES', 'MAPS_AND_NAVIGATION'],
      dtype='object')

In [18]:
# preformat content_rating
df, _ = categorize(df, 'content_rating')
df['content_rating'].unique()
categories['content_rating']

array([0, 1, 2, 3, 4, 5])

Index(['Everyone', 'Teen', 'Everyone 10+', 'Mature 17+', 'Adults only 18+',
       'Unrated'],
      dtype='object')

In [19]:
# preformat genres
# there are 119 unique genres
print('Unique genres before preprocessing : %d' % df['genres'].nunique())

# keep only the first genre
df['genres'].value_counts().tail(10)

# there are 498 apps that have two genres
print(' Apps with more than one genre : %d ' % df['genres'].str.contains(';').sum())

df['genres'] = df['genres'].str.split(';').str[0]

df, _ = categorize(df, 'genres')
# we are down to 48 unique genres
print('Unique genres : %d' % df['genres'].nunique())
df['genres'].unique()
categories['genres']

Unique genres before preprocessing : 118


Strategy;Creativity                    1
Health & Fitness;Action & Adventure    1
Casual;Music & Video                   1
Racing;Pretend Play                    1
Tools;Education                        1
Books & Reference;Creativity           1
Health & Fitness;Education             1
Parenting;Brain Games                  1
Adventure;Education                    1
Role Playing;Brain Games               1
Name: genres, dtype: int64

 Apps with more than one genre : 393 
Unique genres : 48


array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47])

Index(['Art & Design', 'Auto & Vehicles', 'Beauty', 'Books & Reference',
       'Business', 'Comics', 'Communication', 'Dating', 'Education',
       'Entertainment', 'Events', 'Finance', 'Food & Drink',
       'Health & Fitness', 'House & Home', 'Libraries & Demo', 'Lifestyle',
       'Adventure', 'Arcade', 'Casual', 'Card', 'Action', 'Strategy', 'Puzzle',
       'Sports', 'Music', 'Word', 'Racing', 'Simulation', 'Board', 'Trivia',
       'Role Playing', 'Educational', 'Music & Audio',
       'Video Players & Editors', 'Medical', 'Social', 'Shopping',
       'Photography', 'Travel & Local', 'Tools', 'Personalization',
       'Productivity', 'Parenting', 'Weather', 'News & Magazines',
       'Maps & Navigation', 'Casino'],
      dtype='object')

## Datetime features

In [20]:
# preformat last_updated -> convert it to difference in days
df['last_updated'] = pd.to_datetime(df['last_updated'])

# consider the max last_updated in df, the date of reference for the other last_updated 
# last_updated will become a negative integer (the number of days between that date and date of reference)
df['last_updated_days'] = (df['last_updated'].max() - df['last_updated']).dt.days

In [21]:
df['last_updated_year'] = df['last_updated'].dt.year

In [22]:
print('Oldest updated app : %d' % df['last_updated_year'].min())

Oldest updated app : 2010


In [23]:
df['last_updated_month'] = df['last_updated'].dt.month

In [24]:
df['last_updated_day'] = df['last_updated'].dt.day

In [25]:
df['last_updated_year'] = df['last_updated_year'].max() - df['last_updated_year']

In [26]:
df['last_updated_month_sin'] = np.sin((df['last_updated_month']-1)*(2.*np.pi/12))
df['last_updated_month_cos'] = np.cos((df['last_updated_month']-1)*(2.*np.pi/12))

## Feature engineering
Features below are derived from the original features of data

In [27]:
# preprocess name
# keep the word count of the app name
import string

def remove_punctuation(s):
    return s.translate(str.maketrans('', '', string.punctuation))

df['name_wc'] = df['name'].apply(lambda s : len(remove_punctuation(s).split()))

In [28]:
print('Longest app name with %d words : %s' % (df['name_wc'].max(), df['name'][df['name_wc'].idxmax()]))
print('Shortes app name with %d words : %s' % (df['name_wc'].min(), df['name'][df['name_wc'].idxmin()]))


Longest app name with 25 words : 591 housing transactions - renting houses, middle-class houses, new cases, real-time registration, villas through the sky, apartment suites, MRT, buying a house selling prices, housing mortgages
Shortes app name with 1 words : REPUVE


In [29]:
def vs_extract(version_str):
    for char in version_str:
        try:
            version = float(char)
            return version
        except :
            pass
    return np.NaN
# preprocess version & android_version
def vs_transform(version):
    if version.lower() == 'varies with device':
        return np.NaN
    if version == np.NaN or version == np.nan or version == 'nan':
        return np.NaN
#     try :
    version = vs_extract(version.split('.')[0])
#     except :
#         print(version)        
    return version
# there are some edge cases that still need to be cared about
# df['version'].astype(str).sort_values()[-1600:]


# transform this to int 

In [30]:
df['version'] = df['version'].astype(str).apply(vs_transform).astype(float)

In [31]:
df['android_version'] = df['android_version'].astype(str).apply(vs_transform).astype(float)

In [32]:
# drop columns not used
drop_columns = ['name', 'last_updated', 'last_updated_month']
df.drop(columns = drop_columns, inplace = True)

## Missing values

Rating column has 10% missing values. To not lose the data, we try and predict its values using the other features.

In [33]:
# check for null values
# rating has a few
# versions have a few nulls also
df.isnull().sum()

category                     0
rating                    1462
reviews                      0
size                         0
installs                     0
type                         0
price                        0
content_rating               0
genres                       0
version                   1084
android_version            991
price_rounded                0
last_updated_days            0
last_updated_year            0
last_updated_day             0
last_updated_month_sin       0
last_updated_month_cos       0
name_wc                      0
dtype: int64

In [34]:
# get the rows with null ratings out, to predict them later
to_predict_rating = df[df['rating'].isnull()]

In [35]:
df = df.dropna()
df.isnull().sum()

category                  0
rating                    0
reviews                   0
size                      0
installs                  0
type                      0
price                     0
content_rating            0
genres                    0
version                   0
android_version           0
price_rounded             0
last_updated_days         0
last_updated_year         0
last_updated_day          0
last_updated_month_sin    0
last_updated_month_cos    0
name_wc                   0
dtype: int64

In [36]:
len(df)

7110

In [37]:
# save the preprocessed version
ready_df = df.copy()

In [38]:
df = ready_df.copy()

In [39]:
df.shape

(7110, 18)

In [2]:
print('Ignore all the gibberish above ! Dataframe is loaded ! ')

Ignore all the gibberish above ! Dataframe is loaded ! 


In [1]:
def get_data(preprocessed = True):
    if preprocessed:
        return ready_df
    else:
        return orig_df